In [2]:
import pandas as pd
import re

인구데이터 불러오기

In [3]:
data = pd.read_csv('./data_to_preprocess/수정_서울시_성별_연령대별_행정동별_거주인구.csv', usecols=[0, 1, 2, 3])
data.rename(columns={'ADM1_NM': '행정동명', 'SEXDSTN_CD': '성별', 'AGRDE_CD': '연령', 'RSPOP_CNT': '거주인구수'}, inplace=True)

# 행정동에서 '제' 글자 제외 - 홍제동 리팩토링 필요
data['행정동명'] = data['행정동명'].apply(lambda x: re.sub(r'제(\d+)동', r'\1동', x) if '제' in x else x)
data

,행정동명,성별,연령,거주인구수
0,안암동,1,0004,174
1,휘경1동,1,0004,326
2,화곡8동,1,0004,446
3,북가좌1동,1,0004,458
4,가양3동,1,0004,448
...,...,...,...,...
17887,개봉3동,2,I00,1
17888,오류1동,2,I00,3
17889,오류2동,2,I00,5
17890,수궁동,2,I00,2


In [9]:
# 0004, 0509 -> 0대, 1014, 1519 -> 10대로 바꾸기
df = data.copy()
category_mapping = {}
category = ['0대', '10대', '20대', '30대', '40대', '50대', '60대', '70대', '80대', '90대', '기타']
for idx, i in enumerate(df['연령'].unique()):
    for idx2, j in enumerate(category):
        if idx // 2 == idx2:
            category_mapping[i] = j

df['연령'] = df['연령'].map(category_mapping)

In [10]:
# 행정동, 성별, 연령별로 그룹핑하고 합계 계산
df = df.groupby(['행정동명', '성별', '연령']).sum()
df

거주인구수
행정동명 성별 연령        
가락1동 1  0대    3204
        10대   2654
        20대   2638
        30대   4412
        40대   4822
...            ...
흑석동  2  60대   3940
        70대   2364
        80대   1364
        90대    282
        기타       2

[9372 rows x 1 columns]

In [11]:
# 인덱스 만들기
index = df.reset_index()['행정동명'].unique()

# 행정동을 인덱스로 하고, 성별_연령대를 컬럼으로 하는 데이터프레임 만들기
final_data = pd.DataFrame(index=index)
for i, j, k in df.index:
    for idx in final_data.index:
        if i == idx:
            final_data.loc[idx, f'{j}_{k}'] = df.loc[(i,j,k), '거주인구수']

# 정수 형식으로 변환
final_data = final_data.astype('int')

In [14]:
final_data.head(5)

,1_0대,1_10대,1_20대,1_30대,1_40대,1_50대,1_60대,1_70대,1_80대,1_90대,...,2_10대,2_20대,2_30대,2_40대,2_50대,2_60대,2_70대,2_80대,2_90대,2_기타
가락1동,3204,2654,2638,4412,4822,3418,3394,1776,366,26,...,2614,2742,5218,4892,3948,3832,1672,478,94,4
가락2동,2156,3486,3988,4442,4994,4904,4076,1868,598,64,...,3078,4208,4726,5304,5224,4686,1834,840,182,8
가락본동,1434,1646,3624,4530,3630,4058,3938,1642,574,64,...,1624,3630,4438,3520,4500,4002,1718,768,132,4
가리봉동,98,132,846,908,265,929,505,405,134,10,...,108,833,503,190,669,397,443,233,40,205
가산동,640,620,6356,7718,3426,3204,2818,1236,436,26,...,668,7668,5174,2188,2292,2392,1252,532,68,4


In [15]:
final_data.to_csv('./preprocessed_data/행정동별 성별 연령대별 거주인구_test.csv')

In [38]:
# # 컬럼 만들기
# gender = [1, 2]
# age = ['0대', '10대', '20대', '30대', '40대', '50대', '60대', '70대', '80대', '90대', '기타']

# columns = []
# for i in gender:
#     for j in age:
#         tmp = f'{i}_{j}'
#         columns.append(tmp)